In [3]:
# suppressMessages(library(rwwa))
invisible(sapply(list.files("~/wwa/rwwa", full.names = T), source))

ERROR: Error in library("extRemes"): there is no package called ‘extRemes’


---
# Fit linear model to gridded data

In [7]:
# load covariate data
gmst = read.table("gmst.txt", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2025]

# define factual & counterfactual covariates
cov_2024 <- gmst[gmst$year == 2025,"gmst", drop = F]
cov_cf <- rbind("pi" = cov_2024 - 1.3)

In [8]:
# loop over all flattened data & fit nonstationary model to each - NB no bootstrapping
fl = list.files("flattened", full.names = T)
for(fnm in fl) {
    
    res_fnm <- gsub("flattened", "flattened-res", fnm)
    
    if (!file.exists(res_fnm)) {
        
        # load the .csv and 
        df <- read.csv(fnm, skip = 3, header = F)
        colnames(df)[1] <- "year"
        df <- merge(gmst, df)

        # fit nonstationary model to all columns
        models <- sapply(colnames(df)[grepl("V", colnames(df))], function(cnm) {
            df_c <- df[,c("year", "gmst", cnm)]
            tryCatch({
                fit_ns("gev", "shift", df_c, cnm, c("gmst"), lower = F)
            }, error = function(cond) { return(NULL) })
        }, simplify = F)

        # get all model results & save as .csv to be reconstructed & plotted in python
        fitted_res <- sapply(models, function(mdl) {
            if(!is.null(unlist(mdl)[1])) {
                mdl_ests(mdl, cov_f = cov_2024, cov_cf = cov_cf)
            } else {
                rep(NA, 11)
            }})
        write.csv(fitted_res, res_fnm)
    }
}